In [3]:
import requests

In [4]:
class Reg_API:
    def __init__(self, page_size=20):
        self.apibase = "https://api.regulations.gov/v4"
        self.apikey = "e7LVpmbLfa0f0dDAx6TPzg86cG5TASGTafkxQHWg"
        self.page_size = page_size
        self.reqstr = ""

    def _add_apikey(self):
        self.reqstr = self.reqstr + f"?api_key={self.apikey}"
        return self

    ### Main url constructors
    def endpoint(self, endpoint):
        self.reqstr = f"{self.apibase}{endpoint}"
        self._add_apikey()
        return self

    def url(self, url):
        self.reqstr = url
        self._add_apikey()
        return self

    ### Search modifiers
    def search(self, search_term):
        self.reqstr = self.reqstr + f"&filter[searchTerm]={search_term}"
        return self

    def page(self, page):
        self.reqstr = self.reqstr + f"&page[size]={self.page_size}&page[number]={page}"
        return self

    ### Get response(s)
    def get(self, get_json=True):
        response = requests.get(self.reqstr)
        if get_json:
            response = response.json()
        self.reqstr = ""
        return response

    def get_all(self):
        responses = []
        page = 1
        while True:
            response = requests.get(self.page(page).reqstr).json()
            responses.append(response)
            if not response["meta"]["hasNextPage"]:
                break
        self.reqstr = ""
        return responses

In [42]:
def get_comment_data(comment):
    return api.url(comment["links"]["self"]).get()["data"]

def get_attachments(comment_data):
    return api.url(comment_data["relationships"]["attachments"]["links"]['related']).get()["data"]

def get_files(attachments):
    return [api.url(data['attributes']["fileFormats"][0]["fileUrl"]).get(get_json=False).content for data in attachments]

def save_attachment(attachment, name):
    with open(f"{name}.pdf", "wb") as f:
        f.write(attachment)

In [56]:
def get_all_comment_data(comment):
    print("comment: ", list(f"{key}: {type(val)}" for key, val in comment.items()))
    comment_data = get_comment_data(comments["data"][0])
    print("comment_data: ", list(f"{key}: {type(val)}" for key, val in comment_data.items()))
    attachments = get_attachments(comment_data)
    print("attachments: ", [f"" list(f"{key}: {type(val)}" for key, val in attachments.items())])
files = get_files(attachments)
    #print(comment_data, "\n\n", attachments, "\n\n", files)

In [74]:
def print_key_types(obj, indent=0):
    for key, val in obj.items():
        if isinstance(val, dict):
            print(" "*indent, f"{key}: {print_key_types(val, indent+2)}")
        else:
            print(" "*indent, f"{key}: {type(val)}")

In [80]:
api = Reg_API()
comments = api.endpoint("/comments").search("climate").get()
sample_comment = comments["data"][0]

print_key_types(sample_comment)

comment_data = get_comment_data(sample_comment)
print("\n")
print_key_types(comment_data)

attachments = get_attachments(comment_data)
print("\n")
for attachment in attachments:
    print_key_types(attachment)

files = get_files(attachments)

 id: <class 'str'>
 type: <class 'str'>
   documentType: <class 'str'>
   lastModifiedDate: <class 'str'>
   highlightedContent: <class 'str'>
   withdrawn: <class 'bool'>
   agencyId: <class 'str'>
   title: <class 'str'>
   objectId: <class 'str'>
   postedDate: <class 'str'>
 attributes: None
   self: <class 'str'>
 links: None


 id: <class 'str'>
 type: <class 'str'>
   self: <class 'str'>
 links: None
   commentOn: <class 'str'>
   commentOnDocumentId: <class 'str'>
   duplicateComments: <class 'int'>
   address1: <class 'NoneType'>
   address2: <class 'NoneType'>
   agencyId: <class 'str'>
   city: <class 'NoneType'>
   category: <class 'NoneType'>
   comment: <class 'str'>
   country: <class 'NoneType'>
   displayProperties: <class 'NoneType'>
   docAbstract: <class 'NoneType'>
   docketId: <class 'str'>
   documentType: <class 'str'>
   email: <class 'NoneType'>
   fax: <class 'NoneType'>
   field1: <class 'NoneType'>
   field2: <class 'NoneType'>
   fileFormats: <class 'NoneT

Parse `["links"]["self"]` to get to comment data

### Main Data

- comment: *comment text*

### Fields to consider

Categories

 - **address info**: address1, address2, city, stateProvinceRegion, country, zip
 - **date info**: modifyDate, postedDate, postmrkDate, receiveDate
 - **submitter info**: firstName, lastName, organization, govAgency, govAgencyType

Other

- title
- commentOnDocumentId
- duplicateComments
- category, documentType, subtype: *agency specific*
- docAbstract: *The detailed description of the document. This is an agency configurable field. An agency can configure this field to make it not publicly accessible*
- docketId

Parse `["relationships"]["attachments"]["links"]['related']` to get to attachment data if needed